In [1]:
import pandas as pd
import numpy as np
import logging, sys
from datetime import datetime
timestamp = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
logging.basicConfig(filename=f'log/jp_log_{timestamp}.log', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

In [ ]:
# main_df = pd.read_csv('data/pandora_to_big5.csv')
# main_df.drop(columns=['#AUTHID'], inplace=True)
# cols = ['cOPN', 'cEXT', 'cNEU', 'cAGR', 'cCON']
# for col in cols:
#     mean_value = main_df[col].mean()
#     main_df[f'{col}'] = main_df[col] > mean_value
#     main_df[cols] = main_df[cols].replace({True: 1, False: 0})  
# liwc_df = pd.read_csv('data/LIWC_pandora_to_big5_oct_24.csv')
# liwc_df = liwc_df.drop(['Unnamed: 0', '#AUTHID', 'STATUS', 'cOPN', 'cEXT', 'cNEU', 'cAGR', 'cCON', ], axis=1)
# df = pd.concat([main_df, liwc_df], axis=1)
# logging.info(f'Merged main and liwc files, Shape:{df.shape}')

# cols = ['cOPN', 'cEXT', 'cNEU', 'cAGR', 'cCON']
# main_df = pd.read_csv('data/mypersonality.csv', encoding='Windows-1252')
# main_df.drop(columns=['#AUTHID', 'sEXT', 'sNEU', 'sAGR', 'sCON', 'sOPN', 'DATE', 'NETWORKSIZE', 'BETWEENNESS', 'NBETWEENNESS', 'DENSITY', 'BROKERAGE', 'NBROKERAGE','TRANSITIVITY'], inplace=True)
# main_df[['cOPN', 'cEXT', 'cNEU', 'cAGR', 'cCON']] = main_df[['cOPN', 'cEXT', 'cNEU', 'cAGR', 'cCON']].replace({'y': 1, 'n': 0})
# liwc_df = pd.read_csv('data/LIWC_mypersonality_oct_2.csv')
# liwc_df = liwc_df.drop(['Unnamed: 0', '#AUTHID', 'ColumnID', 'STATUS'], axis=1)
# df = pd.concat([main_df, liwc_df], axis=1)
# logging.info(f'Merged main and liwc files, Shape:{df.shape}')

In [2]:
from PersonaClassifier import My_training_class
my_train = My_training_class('data/pandora_processed_train.csv', model_list=['mlp'], embedding_model='roberta-base')
# my_train.process_raw_files('data/pandora_to_big5.csv', 'data/LIWC_pandora_to_big5_oct_24.csv' , False)
my_train.preprocess_data()
my_train.df.shape

/home/jmaharja/anaconda3/envs/gpu2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


(44976, 139)

In [4]:
import torch.optim as optim
import torch.nn as nn
import torch
from transformers import AutoTokenizer, AutoModel, get_linear_schedule_with_warmup
def train(model, train_loader, val_loader, max_grad_norm=1.0, epochs=20, lr=0.001):
    logging.info(f'{model.__class__.__name__}; lr={lr}')
    criterion = nn.BCEWithLogitsLoss()  
    optimizer = optim.Adam(model.parameters(), lr=lr)
    total_steps = len(train_loader) * epochs
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)
    for epoch in range(epochs):
        model.train()
        total_loss = 0.0
        for t, labels in train_loader:
            optimizer.zero_grad()  
            outputs= model(t)
            loss = criterion(outputs.squeeze(), labels)
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
            loss.backward()
            optimizer.step()
            scheduler.step()
            total_loss += loss.item()
        model.eval()  
        # print(outputs.shape, labels.shape, outputs.squeeze().shape)
        val_preds, val_labels, val_scores, val_loss = [], [], [], 0
        with torch.no_grad():  
            for inputs, labels in val_loader:
                outputs= model(inputs)
                # print(outputs.shape, labels.shape, outputs.squeeze().shape)
                val_loss += criterion(outputs.squeeze(), labels).item()  
                val_preds.append(torch.sigmoid(outputs))  
                val_labels.append(labels) 
                val_scores.append(outputs)
        val_preds = torch.cat(val_preds)
        val_labels = torch.cat(val_labels)
        val_preds = (val_preds > 0.5).float() 
        val_accuracy = accuracy_score(val_labels.numpy(), val_preds.numpy())
        if epoch % 4 == 0:
            logging.info(f'Epoch [{epoch + 1}/{epochs}], Train Loss: {total_loss / len(train_loader):.4f}, 'f'Val Loss: {val_loss / len(val_loader):.4f}, Val Accuracy: {val_accuracy:.4f}')
    return val_accuracy, val_preds, torch.cat(val_scores)

In [3]:
my_train.df.shape

(44976, 139)

In [25]:
from sklearn.feature_selection import SelectFromModel, SelectKBest, f_classif, mutual_info_classif, VarianceThreshold
from sklearn.linear_model import LogisticRegression
class FeatureSelection:
    def variance_selection(X, threshold=0.16): #(.8 * (1 - .8))
        logging.info(f'Variance Feature Selection. Threshold used: {threshold}')
        selector = VarianceThreshold(threshold=threshold)
        selected_features = selector.fit_transform(X)
        return  X.columns[selector.get_support()]

    def mutual_info_selection(X, y, threshold=0.001):
        logging.info(f'Mutual Info Feature Selection. Threshold used: {threshold}')
        mutual_info = mutual_info_classif(X, y)
        mutual_info = pd.Series(mutual_info)
        mutual_info.index = X.columns
        ig_df = pd.DataFrame({
            'Feature': X.columns,
            'Information Gain': mutual_info
        }).sort_values(by='Information Gain', ascending=False)
        return ig_df[ig_df['Information Gain'] > threshold]['Feature'].values

    def filter_selection(X, y):
        selector = SelectKBest(score_func=mutual_info_classif, k=10)
        selector.fit(X, y)
        return X.columns[selector.get_support()]

    def hybrid_selection(X, y):
        logging.info(f'Hybrid method combining SelectFromModel and Logistic Regression')
        selector = SelectFromModel(LogisticRegression(penalty="l2", C=0.1))
        X_selected = selector.fit_transform(X, y)
        return X.columns[selector.get_support()]

In [27]:
stat_cols = list (set(my_train.df.columns) - set(['STATUS', 'original', 'cEXT','cNEU', 'cAGR', 'cCON', 'cOPN']) - set(['roberta_embeddings']))
default_selection = FeatureSelection.variance_selection(my_train.df[stat_cols])
len(default_selection)

128

In [28]:
mutual_info_selection = FeatureSelection.mutual_info_selection(my_train.df[stat_cols], my_train.df['cCON'], 0.001)
len(mutual_info_selection)

69

In [ ]:
hybrid_selection = FeatureSelection.hybrid_selection(my_train.df[stat_cols], my_train.df['cCON'])
len(hybrid_selection)

In [41]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, KFold
from torch.utils.data import DataLoader, TensorDataset

def prepare_dataset(my_train, target_col, test_size, batch_size, selected_features):
    scaler = StandardScaler() 
    stat_features_scaled = scaler.fit_transform(my_train.df[selected_features])
    X = np.concatenate([stat_features_scaled, my_train.contextual_embeddings] if my_train.contextual_embeddings is not None else [stat_features_scaled], axis=1)
    y = np.array(my_train.df[[target_col]]) 
    y = y.ravel() 
    logging.info(y.shape)
    my_train.X_train, my_train.X_val, my_train.y_train, my_train.y_val = train_test_split(X, y, test_size=0.1, random_state=42)
    X_train_tensor = torch.tensor(my_train.X_train, dtype=torch.float32)
    y_train_tensor = torch.tensor(my_train.y_train, dtype=torch.float32)
    X_val_tensor = torch.tensor(my_train.X_val, dtype=torch.float32)
    y_val_tensor = torch.tensor(my_train.y_val, dtype=torch.float32)
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
    my_train.train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    my_train.val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    # logging.info(my_train.X_train.shape, my_train.y_train.shape)

In [42]:
from PersonaClassifier import MLP, BiLSTMClassifier, train_val_dl_models
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay, precision_score, recall_score, f1_score, roc_curve, auc

In [53]:
prepare_dataset(my_train, 'cCON', test_size=0.1, batch_size=16, selected_features=default_selection)
my_train.mlp_model = MLP(input_size=my_train.X_train.shape[1], hidden_size=128, output_size=1, dropout_rate=0.3)
mlp_acc, y_pred, y_scores = train(my_train.mlp_model, my_train.train_loader, my_train.val_loader, max_grad_norm=1.0, epochs=16, lr=0.001)

In [54]:
prepare_dataset(my_train, 'cCON', test_size=0.1, batch_size=16, selected_features=mutual_info_selection)
my_train.mlp_model = MLP(input_size=my_train.X_train.shape[1], hidden_size=128, output_size=1, dropout_rate=0.3)
mlp_acc, y_pred, y_scores = train(my_train.mlp_model, my_train.train_loader, my_train.val_loader, max_grad_norm=1.0, epochs=16, lr=0.001)

In [58]:
prepare_dataset(my_train, 'cCON', test_size=0.1, batch_size=16, selected_features=hybrid_selection)
my_train.mlp_model = MLP(input_size=my_train.X_train.shape[1], hidden_size=128, output_size=1, dropout_rate=0.5)
mlp_acc, y_pred, y_scores = train(my_train.mlp_model, my_train.train_loader, my_train.val_loader, max_grad_norm=1.0, epochs=16, lr=0.001)

In [62]:
combined_features = FeatureSelection.hybrid_selection(my_train.df[mutual_info_selection], my_train.df['cCON'])
prepare_dataset(my_train, 'cCON', test_size=0.1, batch_size=16, selected_features=combined_features)
my_train.mlp_model = MLP(input_size=my_train.X_train.shape[1], hidden_size=128, output_size=1, dropout_rate=0.5)
mlp_acc, y_pred, y_scores = train(my_train.mlp_model, my_train.train_loader, my_train.val_loader, max_grad_norm=1.0, epochs=16, lr=0.001)

/home/jmaharja/anaconda3/envs/gpu2/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [61]:
len(combined_features)

50

In [ ]:
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, dropout_rate=0.5):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, output_size)
        self.dropout = nn.Dropout(dropout_rate) 
        
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc3(x)
        return x

class DotProductAttention(nn.Module):
    def __init__(self, hidden_dim):
        super(DotProductAttention, self).__init__()
    def forward(self, x):
        query = x[:, -1:, :]  # Shape: (batch_size, 1, hidden_dim * 2)
        scores = torch.bmm(query, x.transpose(1, 2))  # Shape: (batch_size, 1, seq_len)
        attention_weights = torch.softmax(scores, dim=-1)  # Shape: (batch_size, seq_len, 1)
        context_vector = torch.bmm(attention_weights, x)  # Shape: (batch_size, 1, hidden_dim * 2)
        return context_vector, attention_weights

class BiLSTMClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers, bidirectional=True, do_attention=True, dropout_rate=0.5):
        super(BiLSTMClassifier, self).__init__()
        self.do_attention = do_attention
        self.attention = DotProductAttention(hidden_dim)
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, bidirectional=bidirectional, batch_first=True, dropout=dropout_rate)
        self.fc = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)
        self.layer_norm1 = nn.LayerNorm(input_dim) 
        self.layer_norm2 = nn.LayerNorm(hidden_dim * 2)  
        self.dropout = nn.Dropout(dropout_rate)  
    def forward(self, x):
        if len(x.size()) == 2:
            x = x.unsqueeze(1)  
        if self.do_attention:
            context_vector, attention_weights = self.attention(x)
            context_vector = self.layer_norm1(context_vector)
        else:
            context_vector = x
        lstm_output, _ = self.lstm(context_vector)     
        lstm_output = self.layer_norm2(lstm_output)
        last_hidden_state = lstm_output[:, -1, :]  # Shape: (batch_size, hidden_dim * 2)
        last_hidden_state = self.dropout(last_hidden_state)
        output = self.fc(last_hidden_state)  
        return output

In [ ]:
model_filepath = "checkpoint/MLP_2024-11-22_13-17-32.pth"
model = MLP(input_size=903, hidden_size=128, output_size=1)
model.load_state_dict(torch.load(model_filepath))
model.eval()
print(f"Model loaded from {model_filepath}")

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
model.eval()
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
with torch.no_grad():  
    outputs = model(X_test_tensor)
outputs = torch.sigmoid(outputs)
preds = (outputs > 0.5).float()
preds_np = preds.numpy()
y_test_np = y_test  # If y_test is already in numpy format, otherwise y_test.numpy()
accuracy = accuracy_score(y_test_np, preds_np)
precision = precision_score(y_test_np, preds_np)
recall = recall_score(y_test_np, preds_np)
f1 = f1_score(y_test_np, preds_np)

# Print metrics
print(f"Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}")

## Ensemble Training

In [ ]:
train_df, test_df = train_test_split(dff, test_size=0.1, random_state=42)
print(f'Train set size: {train_df.shape}')
print(f'Test set size: {test_df.shape}')all_cols = dff.columns

# label_cols = ['cEXT','cNEU', 'cAGR', 'cCON', 'cOPN']
label_cols = ["cCON"]
remove_cols = ['#AUTHID', 'STATUS', 'sEXT', 'sNEU', 'sAGR', 'sCON', 'sOPN', 'cEXT','cNEU', 'cAGR', 'cCON', 'cOPN', 'DATE']
emb_cols = ['bert_embeddings', 'berttweet_embeddings', 'xlnet_embeddings', 'roberta_embeddings']
stat_cols = list (set(all_cols) - set(remove_cols) - set(emb_cols))

stat_features = train_df[stat_cols]
bert_embeddings = np.array(train_df["bert_embeddings"].tolist())
roberta_embeddings = np.array(train_df["roberta_embeddings"].tolist())

scaler = StandardScaler()  
stat_features_scaled = scaler.fit_transform(stat_features)
X1 = np.concatenate([stat_features_scaled, bert_embeddings], axis=1)
X2 = np.concatenate([stat_features_scaled, roberta_embeddings], axis=1)
y = np.array(train_df[label_cols]) 

# Split data into train+val and test sets (80% train+val, 20% test)
X1_train, X1_val, y1_train, y1_val = train_test_split(X1, y, test_size=0.1, random_state=42)
X2_train, X2_val, y2_train, y2_val = train_test_split(X2, y, test_size=0.1, random_state=42)

train_dataset1 = TensorDataset(torch.tensor(X1_train, dtype=torch.float32), torch.tensor(y1_train, dtype=torch.float32))
val_dataset1 = TensorDataset(torch.tensor(X1_val, dtype=torch.float32), torch.tensor(y1_val, dtype=torch.float32))
train_dataset2 = TensorDataset(torch.tensor(X2_train, dtype=torch.float32), torch.tensor(y2_train, dtype=torch.float32))
val_dataset2 = TensorDataset(torch.tensor(X2_val, dtype=torch.float32), torch.tensor(y2_val, dtype=torch.float32))
train_loader1 = DataLoader(train_dataset1, batch_size=32, shuffle=True)
val_loader1 = DataLoader(val_dataset1, batch_size=32, shuffle=False)
train_loader2 = DataLoader(train_dataset2, batch_size=32, shuffle=True)
val_loader2 = DataLoader(val_dataset2, batch_size=32, shuffle=False)

In [ ]:
#Model Aveaging
model1 = BiLSTMClassifier(input_dim=X2_train.shape[1], hidden_dim=128, output_dim=1, num_layers=2, bidirectional=True, do_attention=True, dropout_rate=0.3)
model2 = BiLSTMClassifier(input_dim=X2_train.shape[1], hidden_dim=128, output_dim=1, num_layers=2, bidirectional=True, do_attention=True, dropout_rate=0.3)

# model1 = MLP(input_size=X1_train.shape[1], hidden_size=128, output_size=1)
# model2 = MLP(input_size=X2_train.shape[1], hidden_size=128, output_size=1)
criterion = nn.BCEWithLogitsLoss()  
optimizer1 = optim.Adam(model1.parameters(), lr=0.001)
optimizer2 = optim.Adam(model2.parameters(), lr=0.001)
num_epochs = 12
max_grad_norm = 1.0

for epoch in range(num_epochs):
    model1.train()
    model2.train()
    total_loss = 0.0

    for (t1, labels1), (t2, labels2) in zip(train_loader1, train_loader2):
        optimizer1.zero_grad()
        optimizer2.zero_grad()
        outputs1 = model1(t1)
        outputs2 = model2(t2)
        
        avg_outputs = (outputs1 + outputs2) / 2.0
        loss = criterion(avg_outputs, labels1)  
        torch.nn.utils.clip_grad_norm_(model1.parameters(), max_grad_norm)
        torch.nn.utils.clip_grad_norm_(model2.parameters(), max_grad_norm)
        loss.backward()
        optimizer1.step()
        optimizer2.step()
        total_loss += loss.item()
    model1.eval()
    model2.eval()
    val_preds = []
    val_labels = []
    val_loss1 = 0
    val_loss2 = 0
    
    with torch.no_grad():
        for (inputs1, labels1), (inputs2, labels2) in zip(val_loader1, val_loader2):
            outputs1 = model1(inputs1)
            outputs2 = model2(inputs2)
            avg_outputs = (outputs1 + outputs2) / 2.0
            val_loss1 += criterion(outputs1, labels1).item()
            val_loss2 += criterion(outputs2, labels2).item()
            val_preds.append(torch.sigmoid(avg_outputs))
            val_labels.append(labels1)
    
    val_preds = torch.cat(val_preds)
    val_labels = torch.cat(val_labels)
    val_preds = (val_preds > 0.5).float()
    val_accuracy = accuracy_score(val_labels.numpy(), val_preds.numpy())
    print(f'Epoch [{epoch + 1}/{num_epochs}], '
          f'Train Loss Model1: {total_loss / len(train_loader1):.4f}, '
          f'Validation Loss Model1: {val_loss1 / len(val_loader1):.4f}, '
          f'Validation Loss Model2: {val_loss2 / len(val_loader2):.4f}, '
          f'Validation Accuracy: {val_accuracy:.4f}')
